## Modeling rain accretion

### Define reference variables

In [1]:
using QuadGK

In [3]:
rho_w = 998
rho_a = 1
g = 9.81
C_d = 0.55
m_e = 3
r_0 = 1e-3
m_0 = 4*pi*rho_w*r_0*r_0*r_0/3
r_0 = 1e-3
n_0 = 16e6
a_0 = pi*r_0*r_0
lambda = 1
v_0 = sqrt((8*g*r_0*(rho_w - rho_a))/(3*C_d*rho_a))

6.886286768247852

In [4]:
q_liq = range(1e-8, 1e-5, 100)

1.0e-8:1.0090909090909091e-7:1.0e-5

In [5]:
f(r) = exp(-lambda*r)*r^(5/2)
I,err = quadgk(f, 0, 2pi, rtol=1e-8)

(3.0461654221614074, 5.1640094336086184e-9)